# Module 05: Google Cloud AI Platform Basics

**Difficulty**: ⭐⭐
**Estimated Time**: 55 minutes
**Prerequisites**: 
- [Module 03: Azure ML Studio Introduction](03_azure_ml_studio_introduction.ipynb)
- Basic understanding of cloud computing concepts
- Familiarity with Python and ML workflows

## Learning Objectives

By the end of this notebook, you will be able to:
1. Set up GCP project and understand billing concepts
2. Use Cloud Storage for ML data management
3. Query and analyze data with BigQuery
4. Work with AI Platform Notebooks for development
5. Submit training jobs to AI Platform
6. Leverage pre-built ML APIs (Vision, Natural Language, etc.)

## 1. Setup and Introduction

### Google Cloud Platform (GCP) Overview

GCP provides comprehensive AI/ML services integrated with data infrastructure.

```
┌─────────────────────────────────────────────────────────┐
│         Google Cloud AI/ML Ecosystem                     │
├─────────────────────────────────────────────────────────┤
│                                                         │
│  Data Storage & Processing                              │
│  ┌──────────────────────────────────────┐              │
│  │ Cloud Storage (objects, files)       │              │
│  │ BigQuery (data warehouse)            │              │
│  │ Cloud SQL (relational DB)            │              │
│  │ Dataflow (stream/batch processing)   │              │
│  └──────────────────────────────────────┘              │
│                    │                                    │
│                    ▼                                    │
│  ML Development                                         │
│  ┌──────────────────────────────────────┐              │
│  │ Vertex AI (unified platform)         │              │
│  │ AI Platform Notebooks (JupyterLab)   │              │
│  │ Colab Enterprise                     │              │
│  └──────────────────────────────────────┘              │
│                    │                                    │
│                    ▼                                    │
│  ML Training & Deployment                               │
│  ┌──────────────────────────────────────┐              │
│  │ Vertex AI Training                   │              │
│  │ Vertex AI Predictions                │              │
│  │ AutoML                               │              │
│  └──────────────────────────────────────┘              │
│                    │                                    │
│                    ▼                                    │
│  Pre-built ML APIs                                      │
│  ┌──────────────────────────────────────┐              │
│  │ Vision API, Natural Language API     │              │
│  │ Translation, Speech-to-Text          │              │
│  │ Recommendations AI                   │              │
│  └──────────────────────────────────────┘              │
└─────────────────────────────────────────────────────────┘
```

**Free Tier**: GCP offers $300 credit for new users (90 days) + always-free tier for many services!

In [ ]:
# Setup and imports
import numpy as np
import pandas as pd
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

In [ ]:
# Mock GCP SDK for demonstration
# In production, use: from google.cloud import storage, bigquery, aiplatform

class MockGCPProject:
    """Simulates GCP project configuration"""
    
    def __init__(self, project_id, region='us-central1'):
        self.project_id = project_id
        self.region = region
        self.services_enabled = []
    
    def enable_service(self, service_name):
        """Enable GCP service API"""
        self.services_enabled.append(service_name)
        print(f"✓ Enabled {service_name}")
    
    def get_info(self):
        """Get project information"""
        return {
            'project_id': self.project_id,
            'region': self.region,
            'services_enabled': self.services_enabled
        }

## 2. GCP Project and Billing Setup

Every GCP resource belongs to a project. Projects organize resources and manage billing.

### Project Hierarchy

```
Organization (optional)
    │
    ├── Folder: Development
    │   ├── Project: ml-dev-001
    │   └── Project: ml-dev-002
    │
    └── Folder: Production
        ├── Project: ml-prod-001
        └── Project: ml-prod-002

Each Project contains:
├── Resources (VMs, Storage, etc.)
├── Billing Account (linked)
├── IAM Permissions
└── Enabled APIs
```

In [ ]:
# Create and configure GCP project

def create_project_config():
    """Define GCP project configuration"""
    config = {
        'project_id': 'my-ml-project-123456',  # Must be globally unique
        'project_name': 'ML Development Project',
        'region': 'us-central1',  # Iowa (cheaper, good for ML)
        'zone': 'us-central1-a',
        'billing_account_id': 'XXXXXX-XXXXXX-XXXXXX',
        'labels': {
            'environment': 'development',
            'team': 'ml-team',
            'cost-center': 'ml-research'
        }
    }
    return config

project_config = create_project_config()
print("GCP Project Configuration:")
print(json.dumps(project_config, indent=2))

# Create mock project
project = MockGCPProject(
    project_id=project_config['project_id'],
    region=project_config['region']
)
print(f"\n✓ Project created: {project.project_id}")

In [ ]:
# Enable required APIs

required_apis = [
    'compute.googleapis.com',           # Compute Engine
    'storage-api.googleapis.com',       # Cloud Storage
    'bigquery.googleapis.com',          # BigQuery
    'aiplatform.googleapis.com',        # Vertex AI
    'notebooks.googleapis.com',         # AI Platform Notebooks
    'vision.googleapis.com',            # Vision API
    'language.googleapis.com',          # Natural Language API
]

print("Enabling GCP APIs:\n")
for api in required_apis:
    project.enable_service(api)

print("\n💡 Tip: Most APIs have free tier quotas before billing!")

In [ ]:
# Budget and billing alerts

def create_budget_alert():
    """Configure budget alerts to control costs"""
    budget_config = {
        'display_name': 'ML Project Monthly Budget',
        'amount': {
            'specified_amount': {
                'currency_code': 'USD',
                'units': 100  # $100/month
            }
        },
        'threshold_rules': [
            {'threshold_percent': 0.50, 'spend_basis': 'CURRENT_SPEND'},  # 50%
            {'threshold_percent': 0.75, 'spend_basis': 'CURRENT_SPEND'},  # 75%
            {'threshold_percent': 0.90, 'spend_basis': 'CURRENT_SPEND'},  # 90%
            {'threshold_percent': 1.00, 'spend_basis': 'CURRENT_SPEND'},  # 100%
        ],
        'notifications': {
            'pubsub_topic': 'projects/my-project/topics/budget-alerts',
            'monitoring_notification_channels': [],
            'disable_default_iam_recipients': False
        }
    }
    
    print("⚠️  Budget Alert Configuration:")
    print(f"Monthly budget: ${budget_config['amount']['specified_amount']['units']}")
    print("Alerts at: 50%, 75%, 90%, 100% of budget")
    print("\n💰 Always set budget alerts to avoid unexpected charges!")
    
    return budget_config

budget = create_budget_alert()

## 3. Cloud Storage for ML Data

Cloud Storage is GCP's object storage service, ideal for ML datasets.

### Storage Architecture

```
┌─────────────────────────────────────────────────────────┐
│           Cloud Storage Structure                        │
├─────────────────────────────────────────────────────────┤
│                                                         │
│  Bucket: gs://my-ml-bucket/                             │
│  ├── data/                                              │
│  │   ├── raw/                                           │
│  │   │   └── sales_2024.csv                             │
│  │   ├── processed/                                     │
│  │   │   └── sales_clean.parquet                        │
│  │   └── features/                                      │
│  │       └── feature_store.tfrecord                     │
│  ├── models/                                            │
│  │   ├── classifier_v1/                                 │
│  │   └── classifier_v2/                                 │
│  └── artifacts/                                         │
│      ├── plots/                                         │
│      └── reports/                                       │
│                                                         │
│  Storage Classes:                                       │
│  - Standard: Frequent access ($0.020/GB/month)          │
│  - Nearline: Monthly access ($0.010/GB/month)           │
│  - Coldline: Quarterly access ($0.004/GB/month)         │
│  - Archive: Yearly access ($0.0012/GB/month)            │
└─────────────────────────────────────────────────────────┘
```

In [ ]:
# Cloud Storage bucket configuration

def create_storage_bucket_config():
    """Configure Cloud Storage bucket for ML data"""
    bucket_config = {
        'name': 'my-ml-data-bucket',  # Globally unique
        'location': 'US-CENTRAL1',  # Same region as compute
        'storage_class': 'STANDARD',  # or NEARLINE, COLDLINE
        'uniform_bucket_level_access': True,  # Recommended
        'versioning': {
            'enabled': True  # Keep file versions
        },
        'lifecycle_rules': [
            {
                'action': {'type': 'SetStorageClass', 'storage_class': 'NEARLINE'},
                'condition': {'age': 30}  # Move to Nearline after 30 days
            },
            {
                'action': {'type': 'Delete'},
                'condition': {'age': 365}  # Delete after 1 year
            }
        ],
        'labels': {
            'purpose': 'ml-data',
            'team': 'ml-team'
        }
    }
    return bucket_config

bucket_config = create_storage_bucket_config()
print("Cloud Storage Bucket Configuration:")
print(json.dumps(bucket_config, indent=2))

In [ ]:
# Simulate Cloud Storage operations

class CloudStorageBucket:
    """Simulates GCP Cloud Storage bucket operations"""
    
    def __init__(self, bucket_name):
        self.bucket_name = bucket_name
        self.blobs = {}  # filename -> metadata
    
    def upload_file(self, local_path, gcs_path):
        """Upload file to Cloud Storage"""
        blob_info = {
            'name': gcs_path,
            'size': np.random.randint(1000, 1000000),  # bytes
            'uploaded_at': datetime.now(),
            'content_type': 'application/octet-stream'
        }
        self.blobs[gcs_path] = blob_info
        print(f"✓ Uploaded {local_path} → gs://{self.bucket_name}/{gcs_path}")
        return gcs_path
    
    def download_file(self, gcs_path, local_path):
        """Download file from Cloud Storage"""
        if gcs_path in self.blobs:
            print(f"✓ Downloaded gs://{self.bucket_name}/{gcs_path} → {local_path}")
            return True
        else:
            print(f"✗ File not found: {gcs_path}")
            return False
    
    def list_files(self, prefix=''):
        """List files with optional prefix filter"""
        matching_blobs = [
            name for name in self.blobs.keys() 
            if name.startswith(prefix)
        ]
        return matching_blobs
    
    def get_uri(self, blob_name):
        """Get GCS URI for file"""
        return f"gs://{self.bucket_name}/{blob_name}"

In [ ]:
# Test Cloud Storage operations
bucket = CloudStorageBucket('my-ml-data-bucket')

# Upload files
print("Uploading files to Cloud Storage:\n")
bucket.upload_file('data/train.csv', 'data/raw/train.csv')
bucket.upload_file('data/test.csv', 'data/raw/test.csv')
bucket.upload_file('models/model.pkl', 'models/v1/model.pkl')

# List files
print("\nFiles in data/raw/:")
for file in bucket.list_files('data/raw/'):
    print(f"  - {bucket.get_uri(file)}")

# Download file
print("\nDownloading model:")
bucket.download_file('models/v1/model.pkl', 'downloaded_model.pkl')

## 4. BigQuery for Data Analysis

BigQuery is a serverless data warehouse for analytics at scale.

### BigQuery Architecture

```
┌─────────────────────────────────────────────────────────┐
│              BigQuery Architecture                       │
├─────────────────────────────────────────────────────────┤
│                                                         │
│  Data Sources                                           │
│  ┌──────────────────────────────────┐                  │
│  │ Cloud Storage (CSV, Parquet)     │                  │
│  │ Cloud SQL                        │                  │
│  │ Streaming (Dataflow)             │                  │
│  │ External tables                  │                  │
│  └──────────────────────────────────┘                  │
│                │                                        │
│                ▼                                        │
│  BigQuery Dataset                                       │
│  ┌──────────────────────────────────┐                  │
│  │ Tables (structured data)         │                  │
│  │ - Partitioned (by date/time)     │                  │
│  │ - Clustered (by columns)         │                  │
│  │ - Nested/repeated fields         │                  │
│  └──────────────────────────────────┘                  │
│                │                                        │
│                ▼                                        │
│  SQL Queries                                            │
│  ┌──────────────────────────────────┐                  │
│  │ Standard SQL                     │                  │
│  │ ML functions (BQML)              │                  │
│  │ Federated queries                │                  │
│  └──────────────────────────────────┘                  │
│                                                         │
│  Pricing:                                               │
│  - Storage: $0.020/GB/month (first 10GB free)          │
│  - Queries: $5/TB processed (first 1TB/month free)      │
└─────────────────────────────────────────────────────────┘
```

In [ ]:
# BigQuery dataset and table configuration

def create_bigquery_dataset():
    """Configure BigQuery dataset"""
    dataset_config = {
        'dataset_id': 'ml_analytics',
        'project_id': 'my-ml-project-123456',
        'location': 'US',  # Multi-region for better availability
        'default_table_expiration_ms': 90 * 24 * 60 * 60 * 1000,  # 90 days
        'description': 'ML analytics and feature engineering',
        'labels': {
            'team': 'ml-team',
            'purpose': 'analytics'
        }
    }
    return dataset_config

def create_table_schema():
    """Define table schema for customer data"""
    schema = [
        {'name': 'customer_id', 'type': 'STRING', 'mode': 'REQUIRED'},
        {'name': 'purchase_date', 'type': 'DATE', 'mode': 'REQUIRED'},
        {'name': 'amount', 'type': 'FLOAT64', 'mode': 'REQUIRED'},
        {'name': 'product_category', 'type': 'STRING', 'mode': 'NULLABLE'},
        {'name': 'metadata', 'type': 'JSON', 'mode': 'NULLABLE'}
    ]
    
    table_config = {
        'table_id': 'customer_transactions',
        'schema': schema,
        'time_partitioning': {
            'type': 'DAY',
            'field': 'purchase_date'  # Partition by date
        },
        'clustering_fields': ['product_category', 'customer_id']
    }
    return table_config

dataset = create_bigquery_dataset()
table = create_table_schema()

print("BigQuery Dataset:")
print(json.dumps(dataset, indent=2))
print("\nTable Schema:")
print(json.dumps(table, indent=2))

In [ ]:
# Simulate BigQuery operations

class BigQueryClient:
    """Simulates BigQuery client operations"""
    
    def __init__(self, project_id):
        self.project_id = project_id
        self.query_history = []
    
    def query(self, sql):
        """
        Execute SQL query and return results
        Simulates query execution with random data
        """
        # Record query
        query_info = {
            'sql': sql,
            'executed_at': datetime.now(),
            'bytes_processed': np.random.randint(1000000, 10000000)
        }
        self.query_history.append(query_info)
        
        # Simulate results
        if 'COUNT' in sql.upper():
            results = pd.DataFrame({'count': [np.random.randint(1000, 100000)]})
        elif 'AVG' in sql.upper() or 'SUM' in sql.upper():
            results = pd.DataFrame({
                'metric': [np.random.rand() * 1000]
            })
        else:
            # Return sample rows
            results = pd.DataFrame({
                'customer_id': [f'C{i:04d}' for i in range(5)],
                'amount': np.random.rand(5) * 100,
                'category': np.random.choice(['A', 'B', 'C'], 5)
            })
        
        # Calculate cost (simplified)
        bytes_billed = query_info['bytes_processed']
        tb_processed = bytes_billed / (1024**4)
        cost = tb_processed * 5  # $5 per TB
        
        print(f"Query executed:")
        print(f"  Bytes processed: {bytes_billed:,}")
        print(f"  Estimated cost: ${cost:.4f}")
        
        return results
    
    def load_from_gcs(self, gcs_uri, table_ref):
        """Load data from Cloud Storage into BigQuery"""
        print(f"Loading data from {gcs_uri} into {table_ref}")
        rows_loaded = np.random.randint(1000, 100000)
        print(f"✓ Loaded {rows_loaded:,} rows")
        return rows_loaded

In [ ]:
# Test BigQuery operations
bq = BigQueryClient('my-ml-project-123456')

# Example queries
print("Running BigQuery analysis:\n")

# Count query
sql1 = """
SELECT COUNT(*) as count
FROM `my-ml-project-123456.ml_analytics.customer_transactions`
WHERE purchase_date >= '2024-01-01'
"""
result1 = bq.query(sql1)
print(result1)
print()

# Aggregation query
sql2 = """
SELECT 
  customer_id,
  SUM(amount) as total_amount,
  COUNT(*) as num_purchases
FROM `my-ml-project-123456.ml_analytics.customer_transactions`
GROUP BY customer_id
LIMIT 5
"""
result2 = bq.query(sql2)
print(result2)

## 5. AI Platform Notebooks

Managed JupyterLab environment for ML development.

### Notebook Instance Types

```
┌─────────────────────────────────────────────────────────┐
│         AI Platform Notebooks Options                    │
├─────────────────────────────────────────────────────────┤
│                                                         │
│  1. Pre-configured Environments                         │
│     ┌──────────────────────────────┐                   │
│     │ Python 3 (basic)             │                   │
│     │ TensorFlow Enterprise        │                   │
│     │ PyTorch                      │                   │
│     │ R                            │                   │
│     └──────────────────────────────┘                   │
│                                                         │
│  2. Machine Types                                       │
│     ┌──────────────────────────────┐                   │
│     │ n1-standard-4 (4 vCPU)       │  ~$0.19/hour      │
│     │ n1-highmem-8 (8 vCPU)        │  ~$0.47/hour      │
│     │ With GPU (T4, V100)          │  +$0.35-1.00/hr   │
│     └──────────────────────────────┘                   │
│                                                         │
│  3. Features                                            │
│     - JupyterLab interface                              │
│     - Git integration                                   │
│     - Terminal access                                   │
│     - Auto-shutdown (cost saving)                       │
│     - Direct GCS/BigQuery access                        │
└─────────────────────────────────────────────────────────┘
```

In [ ]:
# AI Platform Notebook instance configuration

def create_notebook_instance_config():
    """Configure AI Platform Notebook instance"""
    instance_config = {
        'name': 'ml-development-notebook',
        'machine_type': 'n1-standard-4',  # 4 vCPU, 15 GB RAM
        'accelerator': {
            'type': 'NVIDIA_TESLA_T4',  # Optional GPU
            'core_count': 1
        },
        'environment': {
            'vm_image': {
                'project': 'deeplearning-platform-release',
                'image_family': 'tf-latest-gpu'  # TensorFlow with GPU
            }
        },
        'disk': {
            'boot_disk_type': 'PD_SSD',
            'boot_disk_size_gb': 100
        },
        'idle_shutdown': {
            'enabled': True,
            'idle_timeout': '60m'  # Auto-shutdown after 1 hour
        },
        'labels': {
            'team': 'ml-team',
            'purpose': 'development'
        }
    }
    
    print("💰 Cost Optimization Tips:")
    print("- Enable idle shutdown")
    print("- Use standard (not highmem) VMs when possible")
    print("- Start without GPU, add only when needed")
    print("- Stop instance when not in use\n")
    
    return instance_config

notebook_config = create_notebook_instance_config()
print("AI Platform Notebook Configuration:")
print(json.dumps(notebook_config, indent=2))

## 6. Training Jobs on AI Platform

Submit custom training jobs to managed infrastructure.

### Training Job Flow

```
1. Package training code → 2. Upload to GCS → 3. Submit job
                              ↓
4. AI Platform provisions VMs → 5. Downloads code & data
                              ↓
6. Runs training → 7. Saves model to GCS → 8. Releases VMs
```

In [ ]:
# Training job configuration

def create_training_job_config():
    """Configure custom training job on AI Platform"""
    job_config = {
        'job_id': 'classifier_training_001',
        'training_input': {
            'scaleTier': 'CUSTOM',  # or BASIC, STANDARD_1, PREMIUM_1
            'masterType': 'n1-standard-4',
            'workerCount': 0,  # No workers for single-machine
            'packageUris': [
                'gs://my-ml-bucket/training/trainer-0.1.tar.gz'
            ],
            'pythonModule': 'trainer.task',  # Entry point
            'args': [
                '--train-data', 'gs://my-ml-bucket/data/train.csv',
                '--eval-data', 'gs://my-ml-bucket/data/eval.csv',
                '--job-dir', 'gs://my-ml-bucket/jobs/classifier_001',
                '--epochs', '10',
                '--learning-rate', '0.001'
            ],
            'region': 'us-central1',
            'runtimeVersion': '2.11',  # TensorFlow version
            'pythonVersion': '3.9'
        }
    }
    return job_config

job_config = create_training_job_config()
print("Training Job Configuration:")
print(json.dumps(job_config, indent=2))

In [ ]:
# Simulate training job execution

class TrainingJob:
    """Simulates AI Platform training job"""
    
    def __init__(self, job_id):
        self.job_id = job_id
        self.state = 'PREPARING'
        self.metrics = {}
    
    def submit(self):
        """Submit training job"""
        print(f"Submitting job: {self.job_id}")
        print("State: PREPARING")
        print("Provisioning compute resources...\n")
        self.state = 'RUNNING'
    
    def run_training(self, epochs=5):
        """Simulate training execution"""
        print("Training started\n")
        
        for epoch in range(1, epochs + 1):
            # Simulate metrics
            loss = 1.0 / epoch + np.random.rand() * 0.1
            accuracy = 0.6 + (epoch * 0.05) + np.random.rand() * 0.02
            
            self.metrics[epoch] = {
                'loss': loss,
                'accuracy': accuracy
            }
            
            print(f"Epoch {epoch}/{epochs}: loss={loss:.4f}, accuracy={accuracy:.4f}")
        
        self.state = 'SUCCEEDED'
        print(f"\n✓ Training completed successfully")
        print(f"Model saved to: gs://my-ml-bucket/jobs/{self.job_id}/model/")
    
    def get_status(self):
        """Get job status"""
        return {
            'job_id': self.job_id,
            'state': self.state,
            'final_metrics': self.metrics.get(max(self.metrics.keys())) if self.metrics else None
        }

In [ ]:
# Run training job simulation
job = TrainingJob('classifier_training_001')
job.submit()
job.run_training(epochs=5)

# Check final status
status = job.get_status()
print("\nFinal Status:")
print(json.dumps(status, indent=2, default=str))

## 7. Pre-built ML APIs

GCP offers ready-to-use ML APIs for common tasks.

### Available APIs

```
┌─────────────────────────────────────────────────────────┐
│           Pre-built ML APIs                              │
├─────────────────────────────────────────────────────────┤
│                                                         │
│  Vision API                                             │
│  - Label detection                                      │
│  - Face detection                                       │
│  - OCR (text extraction)                                │
│  - Logo detection                                       │
│                                                         │
│  Natural Language API                                   │
│  - Sentiment analysis                                   │
│  - Entity extraction                                    │
│  - Syntax analysis                                      │
│  - Content classification                               │
│                                                         │
│  Translation API                                        │
│  - 100+ languages                                       │
│  - Auto language detection                              │
│                                                         │
│  Speech-to-Text API                                     │
│  - Real-time transcription                              │
│  - Speaker diarization                                  │
│                                                         │
│  Pricing: Pay per request (free tier available)         │
└─────────────────────────────────────────────────────────┘
```

In [ ]:
# Simulate Vision API

class VisionAPIClient:
    """Simulates Google Cloud Vision API"""
    
    def __init__(self):
        self.api_calls = 0
    
    def detect_labels(self, image_uri):
        """
        Detect labels in image
        Returns: List of labels with confidence scores
        """
        self.api_calls += 1
        
        # Simulated labels
        labels = [
            {'description': 'Cat', 'score': 0.98},
            {'description': 'Pet', 'score': 0.95},
            {'description': 'Mammal', 'score': 0.93},
            {'description': 'Whiskers', 'score': 0.87},
        ]
        
        print(f"Analyzing image: {image_uri}")
        print("\nDetected labels:")
        for label in labels:
            print(f"  {label['description']}: {label['score']:.2%}")
        
        return labels
    
    def detect_text(self, image_uri):
        """OCR text extraction from image"""
        self.api_calls += 1
        
        # Simulated OCR result
        text = "Sample text extracted from image\nMultiple lines supported"
        
        print(f"Extracting text from: {image_uri}")
        print(f"\nExtracted text:\n{text}")
        
        return text

# Test Vision API
vision = VisionAPIClient()
labels = vision.detect_labels('gs://my-bucket/images/cat.jpg')
print("\n" + "="*50 + "\n")
text = vision.detect_text('gs://my-bucket/images/receipt.jpg')

In [ ]:
# Simulate Natural Language API

class NaturalLanguageAPIClient:
    """Simulates Google Cloud Natural Language API"""
    
    def __init__(self):
        self.api_calls = 0
    
    def analyze_sentiment(self, text):
        """
        Analyze sentiment of text
        Returns: Sentiment score and magnitude
        """
        self.api_calls += 1
        
        # Simulated sentiment
        sentiment = {
            'score': np.random.randn() * 0.3,  # -1 to 1
            'magnitude': np.random.rand() * 2  # 0 to inf
        }
        
        print(f"Analyzing text: '{text[:50]}...'")
        print(f"\nSentiment:")
        print(f"  Score: {sentiment['score']:.2f} (negative to positive)")
        print(f"  Magnitude: {sentiment['magnitude']:.2f} (strength)")
        
        if sentiment['score'] > 0.25:
            print(f"  Overall: Positive")
        elif sentiment['score'] < -0.25:
            print(f"  Overall: Negative")
        else:
            print(f"  Overall: Neutral")
        
        return sentiment
    
    def extract_entities(self, text):
        """Extract named entities from text"""
        self.api_calls += 1
        
        # Simulated entities
        entities = [
            {'name': 'Google Cloud', 'type': 'ORGANIZATION', 'salience': 0.8},
            {'name': 'machine learning', 'type': 'OTHER', 'salience': 0.6},
            {'name': 'United States', 'type': 'LOCATION', 'salience': 0.3},
        ]
        
        print(f"Extracting entities from: '{text[:50]}...'")
        print(f"\nEntities found:")
        for entity in entities:
            print(f"  {entity['name']} ({entity['type']}) - salience: {entity['salience']:.2f}")
        
        return entities

# Test Natural Language API
nlp = NaturalLanguageAPIClient()
sentiment = nlp.analyze_sentiment(
    "Google Cloud Platform offers excellent machine learning tools!"
)
print("\n" + "="*50 + "\n")
entities = nlp.extract_entities(
    "Google Cloud provides AI services in data centers across the United States."
)

## Exercises

### Exercise 1: Data Pipeline Design

Design a data pipeline that:
1. Ingests daily CSV files from Cloud Storage
2. Loads into BigQuery with partitioning
3. Runs SQL transformations for features
4. Exports results back to GCS for training

Consider cost optimization and scheduling.

In [ ]:
# Your solution here
def design_data_pipeline():
    """
    Design complete data pipeline for ML
    
    Include:
    - Data ingestion from GCS
    - BigQuery loading and transformations
    - Feature engineering SQL
    - Export for training
    - Scheduling strategy
    
    Returns:
        dict with pipeline specification
    """
    # TODO: Design pipeline
    pass

# Test your pipeline
# pipeline = design_data_pipeline()
# print(json.dumps(pipeline, indent=2))

### Exercise 2: Cost Optimization

You have:
- 100 GB of training data in Cloud Storage
- BigQuery queries processing 5 TB/month
- Notebook instance running 40 hours/week
- 10 training jobs/month (4 hours each, n1-standard-8)

Calculate monthly cost and propose optimizations.

In [ ]:
# Your solution here
def calculate_gcp_costs(usage):
    """
    Calculate GCP costs and suggest optimizations
    
    Args:
        usage: dict with resource usage details
    
    Returns:
        dict with cost breakdown and optimization suggestions
    """
    # TODO: Calculate costs
    # Cloud Storage: $0.020/GB/month
    # BigQuery: $5/TB processed
    # Compute: varies by machine type
    pass

# Test your calculator
# usage = {
#     'storage_gb': 100,
#     'bigquery_tb_per_month': 5,
#     'notebook_hours_per_week': 40,
#     'training_jobs_per_month': 10,
#     'training_hours_per_job': 4
# }
# costs = calculate_gcp_costs(usage)
# print(json.dumps(costs, indent=2))

### Exercise 3: API Integration

Build a content moderation system using:
- Vision API: Detect inappropriate images
- Natural Language API: Analyze text sentiment/toxicity
- Combine results for final decision

Handle rate limits and error cases.

In [ ]:
# Your solution here
class ContentModerationSystem:
    """Content moderation using GCP ML APIs"""
    
    def __init__(self):
        # TODO: Initialize API clients
        pass
    
    def moderate_content(self, image_uri=None, text=None):
        """
        Moderate content using Vision and NLP APIs
        
        Returns:
            dict with moderation decision and reasoning
        """
        # TODO: Implement moderation logic
        pass

# Test your system
# moderator = ContentModerationSystem()
# result = moderator.moderate_content(
#     image_uri='gs://my-bucket/user_upload.jpg',
#     text='User comment text here'
# )
# print(json.dumps(result, indent=2))

## Summary

In this notebook, you learned:

1. **GCP Setup**: Project configuration, billing, and API enablement
2. **Cloud Storage**: Object storage for ML datasets and models
3. **BigQuery**: Serverless data warehouse for analytics at scale
4. **AI Platform Notebooks**: Managed JupyterLab for development
5. **Training Jobs**: Custom training on managed infrastructure
6. **Pre-built APIs**: Vision, Natural Language for common ML tasks

### Key Takeaways

- Always set budget alerts to control costs
- Use Cloud Storage lifecycle policies to optimize storage costs
- Partition and cluster BigQuery tables for query performance
- Enable idle shutdown for notebook instances
- Start with pre-built APIs before building custom models
- Use the free tier and credits for learning

### What's Next?

- [Module 06: GCP Vertex AI](06_gcp_vertex_ai.ipynb)
- Explore Dataflow for real-time data processing
- Practice with BigQuery ML for in-database modeling
- Build end-to-end ML pipeline with GCP services

### Additional Resources

- [GCP Free Tier](https://cloud.google.com/free)
- [BigQuery Best Practices](https://cloud.google.com/bigquery/docs/best-practices)
- [AI Platform Documentation](https://cloud.google.com/ai-platform/docs)
- [ML APIs Pricing](https://cloud.google.com/products/ai)